In [2]:
import pandas as pd
import numpy as np
zwd_df = pd.read_csv('/root/data/rrr/integrated_weather_dataset/data/processed/Troposphere/2004.csv')
guan_df = pd.read_csv('/root/data/rrr/integrated_weather_dataset/data/processed/Guan/2004.csv')
rutz_df = pd.read_csv('/root/data/rrr/integrated_weather_dataset/data/processed/Rutz/2004.csv')

In [92]:
zwd_df["Timestamp"] = pd.to_datetime(zwd_df["Timestamp"])
guan_df["Timestamp"] = pd.to_datetime(guan_df["Timestamp"])
rutz_df["Timestamp"] = pd.to_datetime(rutz_df["Timestamp"])
rutz_df = rutz_df.rename(columns={
    'longitude': 'Longitude', 
    'latitude': 'Latitude',
    'ARs': 'Rutz_AR_Label'
})


In [106]:
zwd_df

,Timestamp,Site,Latitude,Longitude,ZWD
0,2004-01-01 00:00:00.000000,AGMT,34.594282,-116.429377,27.7
1,2004-01-01 00:04:59.180331,AGMT,34.594282,-116.429377,28.3
2,2004-01-01 00:09:58.360654,AGMT,34.594282,-116.429377,28.7
3,2004-01-01 00:14:57.540985,AGMT,34.594282,-116.429377,29.2
4,2004-01-01 00:19:56.721309,AGMT,34.594282,-116.429377,29.6
...,...,...,...,...,...
16203789,2004-12-30 02:38:30.655736,WWMT,33.955313,-116.653855,54.6
16203790,2004-12-30 02:43:29.836067,WWMT,33.955313,-116.653855,54.5
16203791,2004-12-30 02:48:29.016391,WWMT,33.955313,-116.653855,54.5
16203792,2004-12-30 02:53:28.196722,WWMT,33.955313,-116.653855,54.5


In [93]:
LAT_TOL = 0.25
LON_TOL = 0.3125
CLOSEST_TIME_WINDOW = pd.Timedelta(hours=3)

def round(timestamp):
    return pd.Timestamp(timestamp).round('5min')



In [94]:
def precompute_spatial_matches(zwd_df, ar_df, LAT_TOL, LON_TOL):
    spatial_matches = {}
    for _, site_row in zwd_df.iterrows():
        site_id = site_row["Site"]
        lat1, lon1 = site_row["Latitude"], site_row["Longitude"]
        spatial_mask = (
            (ar_df["Latitude"] >= lat1 - LAT_TOL)
            & (ar_df["Latitude"] <= lat1 + LAT_TOL)
            & (ar_df["Longitude"] >= lon1 - LON_TOL)
            & (ar_df["Longitude"] <= lon1 + LON_TOL)
        )
        spatial_matches[site_id] = ar_df[spatial_mask].reset_index(drop=True)
    return spatial_matches


In [97]:
def process_site_day(site, zwd_site_df, ar_site_df, source):
    exact_match_labels = []
    closest_match_labels = []
    ivt_values = [] if source == 'Rutz' else None
    CLOSEST_TIME_WINDOW = pd.Timedelta(hours=3)
    EXACT_TIME_TOLERANCE = pd.Timedelta(minutes=2)
    
    zwd_site_df = zwd_site_df.copy()
    zwd_site_df["Timestamp"] = zwd_site_df["Timestamp"].apply(round)
    zwd_site_df["Timestamp"] = pd.to_datetime(zwd_site_df["Timestamp"])

    for _, zwd_row in zwd_site_df.iterrows():
        rounded_time = zwd_row["Timestamp"]
        exact_label = np.nan
        closest_label = 0
        ivt = np.nan if source == 'Rutz' else None
        
        if not ar_site_df.empty:
            time_difference = abs(ar_site_df["Timestamp"] - rounded_time)
            exact_matches = ar_site_df[time_difference <= EXACT_TIME_TOLERANCE]

            if not exact_matches.empty:
                exact_label = exact_matches.iloc[0][f"{source}_AR_Label"]
                if source == 'Rutz':
                    ivt = exact_matches.iloc[0]["IVT"]
            
            time_diffs = abs(ar_site_df["Timestamp"] - rounded_time)
            time_diffs_within_window = time_diffs[time_diffs <= CLOSEST_TIME_WINDOW]
            if not time_diffs_within_window.empty:
                closest_index = time_diffs_within_window.idxmin()
                closest_match = ar_site_df.loc[closest_index]
                closest_label = closest_match[f"{source}_AR_Label"]
                if source == 'Rutz': 
                    ivt = closest_match["IVT"]
        
        exact_match_labels.append(exact_label)
        closest_match_labels.append(closest_label)
        if source == 'Rutz':
            ivt_values.append(ivt)
    
    zwd_site_df[f"{source}_exact_match_label"] = exact_match_labels
    zwd_site_df[f"{source}_Label"] = closest_match_labels
    if source == 'Rutz':
        zwd_site_df["IVT"] = ivt_values
    
    return zwd_site_df

In [98]:
def process_data(zwd_df, guan_df, rutz_df, LAT_TOL=0.25, LON_TOL=0.3125):
    zwd_sites_df = zwd_df[["Site", "Latitude", "Longitude"]].drop_duplicates()
    
    guan_spatial_matches = precompute_spatial_matches(zwd_sites_df, guan_df, LAT_TOL, LON_TOL)
    rutz_spatial_matches = precompute_spatial_matches(zwd_sites_df, rutz_df, LAT_TOL, LON_TOL)
    
    zwd_df["Day"] = zwd_df["Timestamp"].dt.date
    guan_df["Day"] = guan_df["Timestamp"].dt.date
    rutz_df["Day"] = rutz_df["Timestamp"].dt.date
    
    all_results = []
    for site_id in zwd_sites_df["Site"].unique():
        print(f"Processing site: {site_id}")
        
        zwd_site_df = zwd_df[zwd_df["Site"] == site_id].reset_index(drop=True)
        guan_site_df = guan_spatial_matches.get(site_id, pd.DataFrame())
        rutz_site_df = rutz_spatial_matches.get(site_id, pd.DataFrame())
        
        for day in zwd_site_df["Day"].unique():
            zwd_day_df = zwd_site_df[zwd_site_df["Day"] == day].reset_index(drop=True)
            
            labeled_day_df = process_site_day(site_id, zwd_day_df, guan_site_df, 'Guan')
            labeled_day_df = process_site_day(site_id, labeled_day_df, rutz_site_df, 'Rutz')
            
            all_results.append(labeled_day_df)
            print(f"Processed day: {day} for site: {site_id}")
            
    final_result = pd.concat(all_results, ignore_index=True)
    final_result = final_result.drop(columns=['Day'])
    
    return final_result


In [107]:
import time
start_time = time.time()

df1 = zwd_df[0:100000]
df2 = guan_df[0:100000]
df3 = rutz_df[0:100000]

final_result = process_data(df1, df2, df3)
end_time = time.time()
print("Time taken :", end_time-start_time)

/tmp/ipykernel_547/1181114857.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zwd_df["Day"] = zwd_df["Timestamp"].dt.date
/tmp/ipykernel_547/1181114857.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guan_df["Day"] = guan_df["Timestamp"].dt.date
/tmp/ipykernel_547/1181114857.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

Processing site: AGMT
Processed day: 2004-01-01 for site: AGMT
Processed day: 2004-01-02 for site: AGMT
Processing site: ALAM
Processed day: 2004-01-01 for site: ALAM
Processed day: 2004-01-02 for site: ALAM
Processing site: ALPP
Processed day: 2004-01-01 for site: ALPP
Processed day: 2004-01-02 for site: ALPP
Processing site: AOA1
Processed day: 2004-01-01 for site: AOA1
Processed day: 2004-01-02 for site: AOA1
Processing site: ARGU
Processed day: 2004-01-01 for site: ARGU
Processed day: 2004-01-02 for site: ARGU
Processing site: ARM1
Processed day: 2004-01-01 for site: ARM1
Processed day: 2004-01-02 for site: ARM1
Processing site: ARM2
Processed day: 2004-01-01 for site: ARM2
Processed day: 2004-01-02 for site: ARM2
Processing site: AVRY
Processed day: 2004-01-01 for site: AVRY
Processed day: 2004-01-02 for site: AVRY
Processing site: AZRY
Processed day: 2004-01-01 for site: AZRY
Processed day: 2004-01-02 for site: AZRY
Processing site: BALD
Processed day: 2004-01-01 for site: BALD
P

In [130]:
def process_data_efficient(zwd_df, guan_df, rutz_df, LAT_TOL=0.25, LON_TOL=0.3125):
    zwd_df = zwd_df.copy()
    zwd_df["Timestamp"] = zwd_df["Timestamp"].dt.round('5min')
    
    result_columns = {
        "Guan_exact_match_label": np.nan,
        "Guan_Label": 0,
        "Rutz_exact_match_label": np.nan,
        "Rutz_Label": 0,
        "IVT": np.nan
    }
    for col in result_columns:
        zwd_df[col] = result_columns[col]

    zwd_df = zwd_df.sort_values("Timestamp")
    guan_df = guan_df.sort_values("Timestamp")
    rutz_df = rutz_df.sort_values("Timestamp")
    
    for site in sorted(zwd_df["Site"].unique()):
        print(f"Processing site: {site}")
        
        site_data = zwd_df[zwd_df["Site"] == site].iloc[0]
        lat, lon = site_data["Latitude"], site_data["Longitude"]
        
        guan_spatial_mask = (
            (guan_df["Latitude"] >= lat - LAT_TOL)
            & (guan_df["Latitude"] <= lat + LAT_TOL)
            & (guan_df["Longitude"] >= lon - LON_TOL)
            & (guan_df["Longitude"] <= lon + LON_TOL)
        )
        rutz_spatial_mask = (
            (rutz_df["Latitude"] >= lat - LAT_TOL)
            & (rutz_df["Latitude"] <= lat + LAT_TOL)
            & (rutz_df["Longitude"] >= lon - LON_TOL)
            & (rutz_df["Longitude"] <= lon + LON_TOL)
        )
        
        guan_matches = guan_df[guan_spatial_mask]
        rutz_matches = rutz_df[rutz_spatial_mask]
        
        site_mask = zwd_df["Site"] == site
        site_timestamps = zwd_df.loc[site_mask, "Timestamp"].values
        
        if not guan_matches.empty:
            guan_timestamps = guan_matches["Timestamp"].values
            guan_labels = guan_matches["Guan_AR_Label"].values
            
            time_diffs = np.abs(site_timestamps[:, np.newaxis] - guan_timestamps)
            
            exact_mask = time_diffs <= np.timedelta64(2, 'm')
            exact_indices = np.where(exact_mask.any(axis=1), time_diffs.argmin(axis=1), -1)
            
            closest_mask = time_diffs <= np.timedelta64(3, 'h')
            closest_indices = np.where(closest_mask.any(axis=1), time_diffs.argmin(axis=1), -1)
            
            mask = exact_indices != -1
            zwd_df.loc[site_mask & pd.Series(mask, index=zwd_df[site_mask].index), "Guan_exact_match_label"] = guan_labels[exact_indices[mask]]
            
            mask = closest_indices != -1
            zwd_df.loc[site_mask & pd.Series(mask, index=zwd_df[site_mask].index), "Guan_Label"] = guan_labels[closest_indices[mask]]
        
        if not rutz_matches.empty:
            rutz_timestamps = rutz_matches["Timestamp"].values
            rutz_labels = rutz_matches["Rutz_AR_Label"].values
            rutz_ivt = rutz_matches["IVT"].values
            
            time_diffs = np.abs(site_timestamps[:, np.newaxis] - rutz_timestamps)
            
            exact_mask = time_diffs <= np.timedelta64(2, 'm')
            exact_indices = np.where(exact_mask.any(axis=1), time_diffs.argmin(axis=1), -1)
            
            closest_mask = time_diffs <= np.timedelta64(3, 'h')
            closest_indices = np.where(closest_mask.any(axis=1), time_diffs.argmin(axis=1), -1)
            
            mask = exact_indices != -1
            zwd_df.loc[site_mask & pd.Series(mask, index=zwd_df[site_mask].index), "Rutz_exact_match_label"] = \
                rutz_labels[exact_indices[mask]]
            
            mask = closest_indices != -1
            zwd_df.loc[site_mask & pd.Series(mask, index=zwd_df[site_mask].index), "Rutz_Label"] = \
                rutz_labels[closest_indices[mask]]
            zwd_df.loc[site_mask & pd.Series(mask, index=zwd_df[site_mask].index), "IVT"] = \
                rutz_ivt[closest_indices[mask]]
    
    if 'Day' in zwd_df.columns:
        zwd_df = zwd_df.drop(columns=['Day'])
    
    final_result = zwd_df.sort_values(by=['Timestamp', 'Site']).reset_index(drop=True)
    
    return final_result


In [131]:
start_time = time.time()
final_result_2 = process_data_efficient(df1, df2, df3)
end_time = time.time()
print("time taken:", end_time - start_time)


Processing site: AGMT
Processing site: ALAM
Processing site: ALPP
Processing site: AOA1
Processing site: ARGU
Processing site: ARM1
Processing site: ARM2
Processing site: AVRY
Processing site: AZRY
Processing site: BALD
Processing site: BBDM
Processing site: BBRY
Processing site: BCWR
Processing site: BEAT
Processing site: BEMT
Processing site: BEPK
Processing site: BGIS
Processing site: BILL
Processing site: BKAP
Processing site: BKMS
Processing site: BLSA
Processing site: BMHL
Processing site: BMRY
Processing site: BRAN
Processing site: BRPK
Processing site: BSRY
Processing site: BULL
Processing site: BUST
Processing site: BVPP
Processing site: CA99
Processing site: CACT
Processing site: CASA
Processing site: CAT1
Processing site: CAT2
Processing site: CBHS
Processing site: CCCC
Processing site: CCCO
Processing site: CCCS
Processing site: CDMT
Processing site: CGDM
Processing site: CHIL
Processing site: CHLO
Processing site: CHMS
Processing site: CIC1
Processing site: CIRX
Processing

In [132]:
final_result_2

,Timestamp,Site,Latitude,Longitude,ZWD,Guan_exact_match_label,Guan_Label,Rutz_exact_match_label,Rutz_Label,IVT
0,2004-01-01 00:00:00,AGMT,34.594282,-116.429377,27.7,0.0,0,NaN,0,NaN
1,2004-01-01 00:00:00,ALAM,37.357959,-115.158457,43.2,0.0,0,NaN,0,NaN
2,2004-01-01 00:00:00,ALPP,34.824490,-118.694798,53.0,0.0,0,NaN,0,NaN
3,2004-01-01 00:00:00,AOA1,34.157441,-118.830312,48.4,0.0,0,0.0,0,60.723213
4,2004-01-01 00:00:00,ARGU,36.050011,-117.521942,35.7,0.0,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...
99995,2004-01-02 23:55:00,HOTK,37.658588,-118.821250,6.2,NaN,0,NaN,0,NaN
99996,2004-01-02 23:55:00,HVYS,34.441230,-119.187538,88.9,NaN,0,NaN,1,267.338715
99997,2004-01-02 23:55:00,IID2,32.706170,-115.031802,100.7,NaN,0,NaN,0,NaN
99998,2004-01-02 23:55:00,ISLK,35.662271,-118.474299,33.2,NaN,0,NaN,0,NaN


,Timestamp,Site,Latitude,Longitude,ZWD,Day
0,2004-01-01 00:00:00.000000,AGMT,34.594282,-116.429377,27.7,2004-01-01
1,2004-01-01 00:04:59.180331,AGMT,34.594282,-116.429377,28.3,2004-01-01
2,2004-01-01 00:09:58.360654,AGMT,34.594282,-116.429377,28.7,2004-01-01
3,2004-01-01 00:14:57.540985,AGMT,34.594282,-116.429377,29.2,2004-01-01
4,2004-01-01 00:19:56.721309,AGMT,34.594282,-116.429377,29.6,2004-01-01
...,...,...,...,...,...,...
99995,2004-01-02 05:00:13.934425,JOHN,36.458815,-116.099040,72.8,2004-01-02
99996,2004-01-02 05:05:13.114756,JOHN,36.458815,-116.099040,72.9,2004-01-02
99997,2004-01-02 05:10:12.295079,JOHN,36.458815,-116.099040,73.1,2004-01-02
99998,2004-01-02 05:15:11.475410,JOHN,36.458815,-116.099040,73.4,2004-01-02


In [136]:
import pandas as pd

df = pd.read_csv('/root/data/rrr/integrated_weather_dataset/data/final_icid/2004.csv')
df

,Timestamp,Site,Latitude,Longitude,ZWD,Guan_exact_match_label,Guan_Label,Rutz_exact_match_label,Rutz_Label,IVT
0,2004-07-23 00:00:00,7ODM,34.116407,-117.093192,37.0,0.0,0,0.0,0,60.428555
1,2004-07-23 00:05:00,7ODM,34.116407,-117.093192,36.9,NaN,0,NaN,0,60.428555
2,2004-07-23 00:10:00,7ODM,34.116407,-117.093192,36.7,NaN,0,NaN,0,60.428555
3,2004-07-23 00:15:00,7ODM,34.116407,-117.093192,36.4,NaN,0,NaN,0,60.428555
4,2004-07-23 00:20:00,7ODM,34.116407,-117.093192,36.1,NaN,0,NaN,0,60.428555
...,...,...,...,...,...,...,...,...,...,...
16203789,2004-12-30 02:40:00,WWMT,33.955313,-116.653855,54.6,NaN,1,NaN,0,104.487991
16203790,2004-12-30 02:45:00,WWMT,33.955313,-116.653855,54.5,NaN,1,NaN,0,104.487991
16203791,2004-12-30 02:50:00,WWMT,33.955313,-116.653855,54.5,NaN,1,NaN,0,104.487991
16203792,2004-12-30 02:55:00,WWMT,33.955313,-116.653855,54.5,NaN,1,NaN,0,104.487991


In [139]:
 df.sort_values(by = ["Timestamp","Site"]).reset_index(drop=True)

,Timestamp,Site,Latitude,Longitude,ZWD,Guan_exact_match_label,Guan_Label,Rutz_exact_match_label,Rutz_Label,IVT
0,2004-01-01 00:00:00,AGMT,34.594282,-116.429377,27.7,0.0,0,0.0,0,32.193707
1,2004-01-01 00:00:00,ALAM,37.357959,-115.158457,43.2,0.0,0,0.0,0,64.892807
2,2004-01-01 00:00:00,ALPP,34.824490,-118.694798,53.0,0.0,0,0.0,0,98.817429
3,2004-01-01 00:00:00,AOA1,34.157441,-118.830312,48.4,0.0,0,0.0,0,60.723213
4,2004-01-01 00:00:00,ARGU,36.050011,-117.521942,35.7,0.0,0,0.0,0,86.650131
...,...,...,...,...,...,...,...,...,...,...
16203789,2004-12-30 03:00:00,WMAP,34.259421,-118.414285,94.9,1.0,1,0.0,0,80.507309
16203790,2004-12-30 03:00:00,WNRA,34.043237,-118.059255,104.9,1.0,1,0.0,0,82.323067
16203791,2004-12-30 03:00:00,WOMT,34.669014,-116.931663,37.4,1.0,1,0.0,0,99.418976
16203792,2004-12-30 03:00:00,WRHS,33.958151,-118.427603,98.0,1.0,1,0.0,0,82.323067


In [140]:
df.to_csv('/root/data/rrr/integrated_weather_dataset/data/final_icid/2004.csv')